In [8]:
import pandas as pd
import pyarrow.parquet as pq
from useful_tools import tipo_de_datos

In [9]:
# Cargar las tablas desde los archivos parquet
steam_games = pq.read_table("steam_games.parquet").to_pandas()
users_items = pq.read_table("users_items.parquet").to_pandas()
user_reviews = pq.read_table("user_reviews.parquet").to_pandas()

In [10]:
steam_games

,genres,app_name,release_date,price,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290.0
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400.0
88314,"[Action, Indie, Casual, Sports]",Log Challenge,2012-10-16,2.99,773570.0
...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640.0
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530.0
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660.0
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870.0


In [11]:
user_reviews

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,0
...,...,...,...,...
59300,76561198312638244,70,True,2
59301,76561198312638244,362890,True,2
59302,LydiaMorley,273110,True,2
59303,LydiaMorley,730,True,2


In [12]:
users_items

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,198,10,Counter-Strike,0.001667,0.000000
2,76561197970982479,198,30,Day of Defeat,0.001944,0.000000
8,76561197970982479,198,300,Day of Defeat: Source,1.314722,0.000000
9,76561197970982479,198,240,Counter-Strike: Source,0.514722,0.000000
10,76561197970982479,198,3830,Psychonauts,0.092500,0.000000
...,...,...,...,...,...,...
5153202,76561198329548331,5,304930,Unturned,0.188056,0.188056
5153203,76561198329548331,5,227940,Heroes & Generals,0.011944,0.011944
5153206,76561198329548331,5,388490,One Way To Die: Steam Edition,0.000833,0.000833
5153207,76561198329548331,5,521570,You Have 10 Seconds 2,0.001111,0.001111


In [13]:
def PlayTimeGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games[['id', 'release_date', 'genres']], left_on="item_id", right_on="id", how="left")
    
    # Eliminar la columnas del dataframe resultante
    merged_data.drop(['item_id','item_name','user_id','id', 'items_count', 'playtime_2weeks'], axis=1, inplace=True)
    
    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]
    
    # Convertir la columna "release_date" a tipo datetime
    try:
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)
    except ValueError:
        # Si la inferencia de formato falla, intentar especificar el formato manualmente
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='coerce', format='%b %Y')

    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].dt.year

    # Agrupar por año y calcular la suma de las horas jugadas para cada año
    result = genre_data.groupby('year')['playtime_forever'].sum().reset_index()

    # Encontrar el año con la mayor cantidad de horas jugadas
    max_year = result.loc[result['playtime_forever'].idxmax()]['year']

    return f"El año con más horas jugadas para el género {genre} es: {max_year}"

In [14]:
PlayTimeGenre('Adventure')


C:\Users\fabir\AppData\Local\Temp\ipykernel_18184\3073979638.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)


'El año con más horas jugadas para el género Adventure es: 2014.0'

In [15]:
'''def UserForGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="left")

    # Eliminar la columnas del dataframe resultante
    merged_data.drop(['item_id','price','id','app_name','item_name', 'items_count', 'playtime_2weeks'], axis=1, inplace=True)

    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]
    
    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(genre_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])
    
    # Agrupar por usuario y calcular la suma de las horas jugadas para cada usuario
    user_total_playtime = genre_data.groupby('user_id')['playtime_forever'].sum()
    user_total_playtime = user_total_playtime.reset_index()
    
    # Obtener el usuario con más horas jugadas
    sorted_users = user_total_playtime.sort_values(by='playtime_forever', ascending=False)
    sorted_users = sorted_users.reset_index()
    sorted_users.drop('index', axis=1, inplace=True)
    
    # Obtener el usuario con más horas jugadas
    max_user = sorted_users['user_id'].iloc[0]
    
    # Filtrar los datos solo para el usuario con la máxima cantidad de horas jugadas
    max_user_data = genre_data[genre_data['user_id'] == max_user]

    # Agrupar por año y calcular la suma de las horas jugadas para ese usuario
    year_playtime = max_user_data.groupby('year')['playtime_forever'].sum().reset_index()
    
    # Convertir a lista de tuplas (año, horas acumuladas)
    year_playtime_list = list(year_playtime.to_records(index=False))

    return max_user, year_playtime_list'''

def UserForGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="left")

    # Eliminar la columnas del dataframe resultante
    merged_data.drop(['item_id','price','id','app_name','item_name', 'items_count', 'playtime_2weeks'], axis=1, inplace=True)

    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]
    
    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(genre_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])
    
    # Agrupar por usuario y calcular la suma de las horas jugadas para cada usuario
    user_total_playtime = genre_data.groupby('user_id')['playtime_forever'].sum()
    user_total_playtime = user_total_playtime.reset_index()
    
    # Obtener el usuario con más horas jugadas
    sorted_users = user_total_playtime.sort_values(by='playtime_forever', ascending=False)
    sorted_users = sorted_users.reset_index()
    sorted_users.drop('index', axis=1, inplace=True)
    
    # Obtener el usuario con más horas jugadas
    max_user = sorted_users['user_id'].iloc[0]
    
    # Filtrar los datos solo para el usuario con la máxima cantidad de horas jugadas
    max_user_data = genre_data[genre_data['user_id'] == max_user]

    # Agrupar por año y calcular la suma de las horas jugadas para ese usuario
    year_playtime = max_user_data.groupby('year')['playtime_forever'].sum().reset_index()
    
    # Convertir a lista de listas (año, horas acumuladas)
    year_playtime_list = year_playtime.values.tolist()

    return max_user, year_playtime_list


In [16]:
UserForGenre("Action")

('Sp3ctre',
 [['1995', 0.06027777777777778],
  ['1999', 0.012222222222222223],
  ['2000', 19.623333333333335],
  ['2001', 0.003611111111111111],
  ['2002', 0.0661111111111111],
  ['2003', 2.131388888888889],
  ['2004', 35.39194444444445],
  ['2005', 5.9275],
  ['2006', 0.24888888888888888],
  ['2007', 31.32888888888889],
  ['2008', 0.06222222222222222],
  ['2009', 30.090555555555554],
  ['2010', 21.689722222222223],
  ['2011', 43.026666666666664],
  ['2012', 128.58277777777778],
  ['2013', 33.46138888888889],
  ['2014', 36.30305555555556],
  ['2015', 86.80888888888889],
  ['2016', 8.215555555555556],
  ['2017', 12.035277777777779]])

In [17]:
def UsersRecommend(year):
    # Realizar un left merge entre user_reviews y steam_games
    merged_data = pd.merge(user_reviews[['item_id', 'recommend', 'sentiment_analysis']],
                           steam_games[['id', 'app_name', 'release_date']],
                           left_on='item_id',
                           right_on='id',
                           how='left')
    # Eliminando columnas innecesarias y datos nulos
    merged_data.drop(['id', 'item_id'], axis=1, inplace=True)
    merged_data = merged_data.dropna(subset=['release_date'])
    
    # Filtrar los juegos del año especificado
    # Extraer el año de la columna "release_date"
    merged_data['year'] = merged_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])
    merged_data = merged_data[merged_data['year'] == str(year)]

    # Filtrar los juegos recomendados con sentiment_analysis de 1 o 2
    recommended_games = merged_data[(merged_data['recommend'] == True) & 
                                    (merged_data['sentiment_analysis'].isin([1, 2]))]
    
    # Agrupar por app_name y sumar los valores de sentiment_analysis
    grouped = recommended_games.groupby('app_name')['sentiment_analysis'].sum().reset_index()
    
    # Ordenar los juegos basados en la suma de sentiment_analysis en orden descendente
    recommended_games = grouped.sort_values(by='sentiment_analysis', ascending=False).reset_index().head(3)
    
    # Obtener los nombres de los juegos recomendados
    recommended_games_names = recommended_games['app_name'].tolist()
    
    # Formatear el resultado
    result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(recommended_games_names)]
    
    return result

In [18]:
UsersRecommend(2010)

[{'Puesto 1': "Sid Meier's Civilization® V"},
 {'Puesto 2': 'Mount & Blade: Warband'},
 {'Puesto 3': 'Fallout: New Vegas'}]

In [19]:
def UsersNotRecommend(year):
    # Realizar un left merge entre user_reviews y steam_games
    merged_data = pd.merge(user_reviews[['item_id', 'recommend', 'sentiment_analysis']],
                           steam_games[['id', 'app_name', 'release_date']],
                           left_on='item_id',
                           right_on='id',
                           how='left')
    # Eliminando columnas innecesarias y datos nulos
    merged_data.drop(['id', 'item_id'], axis=1, inplace=True)
    merged_data = merged_data.dropna(subset=['release_date'])
    
    # Filtrar los juegos del año especificado
    # Extraer el año de la columna "release_date"
    merged_data['year'] = merged_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])
    merged_data = merged_data[merged_data['year'] == str(year)]

    # Filtrar los juegos recomendados con sentiment_analysis de 0
    not_recommended_games = merged_data[(merged_data['recommend'] == False) & 
                                    (merged_data['sentiment_analysis'].isin([0]))]
    
    # Agrupar por app_name y sumar los valores de sentiment_analysis
    grouped = not_recommended_games.groupby('app_name').agg(
        sentiment_analysis=('sentiment_analysis', 'sum'),
        count=('sentiment_analysis', 'count')
    ).reset_index()
    
    # Ordenar los juegos basados en la suma de sentiment_analysis en orden descendente
    not_recommended_games = grouped.sort_values(by='count', ascending=False).reset_index().head(3)

    # Obtener los nombres de los juegos recomendados
    not_recommended_games = not_recommended_games['app_name'].tolist()
    
    # Formatear el resultado
    result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(not_recommended_games)]
    
    return result

In [20]:
UsersNotRecommend(2006)

[{'Puesto 1': "Garry's Mod"},
 {'Puesto 2': 'Toribash'},
 {'Puesto 3': '911: First Responders®'}]

In [26]:
def sentiment_analysis(year: str):
    # Agregar la columna "release_date" al dataframe "user_reviews" mediante un merge
    reviews_with_release = pd.merge(user_reviews, steam_games[['id', 'release_date']], 
                                    left_on='item_id', right_on='id', how='left')
    
    # Filtrar las reseñas del año especificado
    reviews_with_release['year'] = reviews_with_release['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(reviews_with_release['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])
    reviews_with_release = reviews_with_release[reviews_with_release['year'] == str(year)]
    
    # Contar las reseñas por categoría de sentimiento
    sentiment_counts = reviews_with_release['sentiment_analysis'].value_counts()
    
    # Convertir los valores de numpy.int32 a int
    sentiment_counts = sentiment_counts.astype(int)
    
    # Crear un diccionario con los conteos de cada categoría de sentimiento
    result = {
        'Negative': int(sentiment_counts.get(0, 0)),
        'Neutral': int(sentiment_counts.get(1, 0)),
        'Positive': int(sentiment_counts.get(2, 0))
    }

    return result


In [27]:
sentiment_analysis(2015)

{'Negative': 1541, 'Neutral': 1398, 'Positive': 3680}